In [1]:
import torch
from PIL import Image
from torchvision import transforms
from clip import clip
from torchvision.datasets import CIFAR10

device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)
cifar10_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [8]:
img, label = cifar10_dataset[0]

In [9]:
img.shape

torch.Size([3, 224, 224])

In [3]:
# car
class_index = 1
# class_images = [i for i in range(len(cifar10_dataset)) if cifar10_dataset[i][1] == class_index]

all_distance = 0
# for image_index in class_images[:10]:
for image_index in range(10):

    image, _ = cifar10_dataset[image_index]
    image = image.unsqueeze(0).to(device)
    image_features1 = model.encode_image(image)

    # Perform rotation (90 degrees in this example)
    rotated_image = transforms.functional.to_pil_image(image.cpu().squeeze(0))
    rotated_image = transforms.functional.rotate(rotated_image, 90)
    rotated_image = transform(rotated_image).unsqueeze(0).to(device)
    image_features2 = model.encode_image(rotated_image)

    app_distance = torch.sum((image_features2 - image_features1)**2)
    #similarity = torch.nn.functional.cosine_similarity(image_features1, image_features2)
    all_distance += app_distance
all_distance / 10

tensor(100.3750, device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)

In [3]:
# cat
class_index = 3
class_images = [i for i in range(len(cifar10_dataset)) if cifar10_dataset[i][1] == class_index]
all_distance1 = 0
for image_index in class_images[:10]:

    image, _ = cifar10_dataset[image_index]
    image = image.unsqueeze(0).to(device)
    image_features1 = model.encode_image(image)

    # Perform rotation (90 degrees in this example)
    rotated_image = transforms.functional.to_pil_image(image.cpu().squeeze(0))
    rotated_image = transforms.functional.rotate(rotated_image, 90)
    rotated_image = transform(rotated_image).unsqueeze(0).to(device)
    image_features2 = model.encode_image(rotated_image)

    app_distance = torch.sum((image_features2 - image_features1)**2)
    #similarity = torch.nn.functional.cosine_similarity(image_features1, image_features2)
    all_distance1 += app_distance
all_distance1 / 10

KeyboardInterrupt: 

In [76]:
# frog
class_index = 6
class_images = [i for i in range(len(cifar10_dataset)) if cifar10_dataset[i][1] == class_index]
all_distance2 = 0
for image_index in class_images[:10]:

    image, _ = cifar10_dataset[image_index]
    image = image.unsqueeze(0).to(device)
    image_features1 = model.encode_image(image)

    # Perform rotation (90 degrees in this example)
    rotated_image = transforms.functional.to_pil_image(image.cpu().squeeze(0))
    rotated_image = transforms.functional.rotate(rotated_image, 90)
    rotated_image = transform(rotated_image).unsqueeze(0).to(device)
    image_features2 = model.encode_image(rotated_image)

    app_distance = torch.sum((image_features2 - image_features1)**2)
    #similarity = torch.nn.functional.cosine_similarity(image_features1, image_features2)
    all_distance2 += app_distance

all_distance2 / 10

tensor(81.6992, grad_fn=<DivBackward0>)

In [30]:
# !pip install torch torchvision sentencepiece
# !pip install git+https://github.com/openai/CLIP.git

import torch
from PIL import Image
from torchvision import transforms
from clip import clip
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
import PIL
from tqdm import tqdm
import argparse

class load_np_dataset(torch.utils.data.Dataset):
    def __init__(self, imgs_path, targets_path, transform):
        self.data = np.load(imgs_path)
        self.targets = np.load(targets_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img , target = self.data[idx], self.targets[idx]
            
        img = PIL.Image.fromarray(img)
        if transform:
            img = self.transform(img)

        return img, target

def parsing():
    parser = argparse.ArgumentParser(description='Tunes a CIFAR Classifier with OE',
                                    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--aug', type=str, default='gaussian_noise',
                        help='select noise.')
    parser.add_argument('--batch_size', '-b', type=int,
                        default=64, help='Batch size.')
    parser.add_argument('--seed', type=int, default=1,
                        help='seed')
    parser.add_argument('--num_workers', type=int, 
                        default=0, help='number of workers')
    parser.add_argument('--transform', type=int, 
                        default=0, help='use transformation dataset')

    args = parser.parse_args()
    return args


In [42]:
import sys
sys.argv = ['']
args = parsing()


device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)


cifar10_path = '/storage/users/makhavan/CSI/finals/datasets/data/'
cifar10_dataset = CIFAR10(root=cifar10_path, train=True, download=True, transform=transform)

if args.transform:
    cifar_train_cor_img_path = f'/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/CIFAR-10-Train-R-A/{args.aug}.npy'
    cifar_train_cor_target_path = '/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/CIFAR-10-Train-R-A/labels-A.npy'
    train_aug_dataset = load_np_dataset(cifar_train_cor_img_path, cifar_train_cor_target_path, transform=transform)
else:
    cifar_train_cor_img_path = f'/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/CIFAR-10-Train-R-C/{args.aug}.npy'
    cifar_train_cor_target_path = '/storage/users/makhavan/CSI/finals/datasets/generalization_repo_dataset/CIFAR-10-Train-R-C/labels-C.npy'
    train_aug_dataset = load_np_dataset(cifar_train_cor_img_path, cifar_train_cor_target_path, transform=transform)

cifar10_loader = DataLoader(cifar10_dataset, shuffle=False, batch_size=args.batch_size, num_workers=args.num_workers)
aug_loader = DataLoader(train_aug_dataset, shuffle=False, batch_size=args.batch_size, num_workers=args.num_workers)


loader = zip(cifar10_loader, aug_loader)
diffs = []


Files already downloaded and verified


In [43]:
for i, data in enumerate(tqdm(loader)):
    data_normal, data_aug = data
    imgs_n, _ = data_normal
    imgs_aug, _ = data_aug
    # imgs_n, imgs_aug = transform(imgs_n).to(device), transform(imgs_aug).to(device)
    imgs_n, imgs_aug = imgs_n.to(device), imgs_aug.to(device)
    break

0it [00:00, ?it/s]


In [46]:
imgs_n_features = model.encode_image(imgs_n)
imgs_aug_features = model.encode_image(imgs_aug)
diffs.append(torch.sum(torch.pow((imgs_n_features - imgs_aug_features), 2), dim=1).detach().cpu().numpy())

In [48]:
import pickle

In [50]:
with open(f'./tensors/diffs_{args.aug}.pkl', 'wb') as f:
    pickle.dump(diffs, f)

In [26]:
import os
import torch
augs = os.listdir('./outputs')
for aug in augs:
    with open(f'./outputs/{aug}', 'r') as file:
        lines = file.readlines()
        lines.pop(0)
    vals = []
    for line in lines:
        vals.append(float(line.split(':')[-1].split('\n')[0]))
    
    vals = torch.tensor(vals)
    vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()
    with open(f'./softmax/{aug}', 'w') as file:
        for val in vals_softmax:
            file.write(str(val)+'\n')

/tmp/ipykernel_10848/2339142470.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  vals_softmax = torch.nn.functional.softmax(vals).detach().cpu().numpy()


In [48]:
augs = os.listdir('./softmax')
augs_probs = {}
for i in range(10):
    augs_probs[i] = {}

for aug in augs:
    with open(f'./softmax/{aug}', 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        # augs_probs[i].append({
        #     aug.split('.')[0] : float(line.split('\n')[0])
        # })
        augs_probs[i][aug.split('.')[0]] = float(line.split('\n')[0])


In [49]:
augs_probs[0]

{'random_crop': 0.0040205405,
 'rot90': 5.038615e-06,
 'gaussian_blur': 0.13409017,
 'gaussian_noise': 0.03901328,
 'motion_blur': 0.03116428,
 'jpeg_compression': 0.090828806,
 'shot_noise': 0.13746814,
 'flip': 0.11583374,
 'brightness': 0.11194416,
 'snow': 0.3783712,
 'fog': 0.4014634,
 'glass_blur': 1.2054588e-08,
 'elastic_transform': 0.06608733,
 'contrast': 0.11341642,
 'speckle_noise': 0.13079225,
 'color_jitter': 0.031809445,
 'saturate': 0.05244525,
 'zoom_blur': 0.04244273,
 'spatter': 0.5304918,
 'rot270': 3.2738772e-06,
 'pixelate': 0.030844223,
 'impulse_noise': 0.0028079278,
 'defocus_blur': 0.13135473}

In [51]:
augs_sorted = {}
for key in augs_probs.keys():
    augs_sorted[key] = {k: v for k, v in sorted(augs_probs[key].items(), key=lambda item: item[1])}

In [57]:
import pickle
with open('./softmax_sorted.pkl', 'wb') as file:
    pickle.dump(augs_sorted, file)

In [59]:
import pickle
with open('./softmax_sorted.pkl', 'rb') as file:
    oo = pickle.load(file)

In [62]:
for key in augs_sorted.keys():
    for key2 in augs_sorted[key].keys():
        print(key2, augs_sorted[key][key2])

glass_blur 1.2054588e-08
rot270 3.2738772e-06
rot90 5.038615e-06
impulse_noise 0.0028079278
random_crop 0.0040205405
pixelate 0.030844223
motion_blur 0.03116428
color_jitter 0.031809445
gaussian_noise 0.03901328
zoom_blur 0.04244273
saturate 0.05244525
elastic_transform 0.06608733
jpeg_compression 0.090828806
brightness 0.11194416
contrast 0.11341642
flip 0.11583374
speckle_noise 0.13079225
defocus_blur 0.13135473
gaussian_blur 0.13409017
shot_noise 0.13746814
snow 0.3783712
fog 0.4014634
spatter 0.5304918
speckle_noise 0.000108593915
shot_noise 0.00057963433
color_jitter 0.0011954387
spatter 0.002020797
impulse_noise 0.002637804
snow 0.010860088
saturate 0.011297863
gaussian_noise 0.01842857
random_crop 0.027475407
pixelate 0.034814823
jpeg_compression 0.04158446
fog 0.048989244
flip 0.08968437
gaussian_blur 0.09388468
defocus_blur 0.09428844
motion_blur 0.09942751
brightness 0.10103441
contrast 0.10612949
zoom_blur 0.1277044
elastic_transform 0.14662114
rot90 0.30168277
rot270 0.4348

In [63]:
import csv

with open('softmax_sorted.csv', 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for key in augs_sorted.keys():
        filewriter.writerow([str(key)])
        for key2 in augs_sorted[key].keys():
            # print(key2, augs_sorted[key][key2])
            filewriter.writerow([str(key2), str(augs_sorted[key][key2])])